# Load packages and set up variables

In [ ]:
if (!require("psych")) rhumba::install("r-psych")
if (!require("dplyr")) rhumba::install("r-dplyr")
if (!require("ggplot2")) rhumba::install("r-ggplot2")
if (!require("car")) rhumba::install("r-car")
if (!require("janitor")) rhumba::install("r-janitor")
if (!require("readxl")) rhumba::install("r-readxl")
if (!require("remotes")) rhumba::install("r-remotes")
#if (!require("hmisc")) rhumba::install("r-hmisc")
if (!require("ggpubr")) rhumba::install("r-ggpubr")
if (!require("ggseg")) install.packages("ggseg")
if (!require("ggseg3d")) install.packages("ggseg3d")
#remotes::install_github("LCBC-UiO/ggsegDKT", auth_token='PLACE TOKEN HERE')
library(ggsegDKT)
if (!require("plotly")) install.packages("plotly")
if (!require("flextable")) install.packages("flextable")
if (!require("Hmisc")) rhumba::install("r-Hmisc")
library('matrixStats')
library(effectsize)
library('mediation')

In [ ]:
plot_mediation_results <- function(data, row_index = 1) {
  # Extract values for the specified row
  result <- data[row_index, ]
  
  ggplot() +
    # Add arrows
    geom_segment(aes(x = 1, xend = 3, y = 2, yend = 2), 
                arrow = arrow(length = unit(0.3, "cm")), size = 0.5) +  # direct effect
    geom_segment(aes(x = 1, xend = 2, y = 2, yend = 3), 
                arrow = arrow(length = unit(0.3, "cm")), size = 0.5) +  # path a
    geom_segment(aes(x = 2, xend = 3, y = 3, yend = 2), 
                arrow = arrow(length = unit(0.3, "cm")), size = 0.5) +  # path b
    
    # Add boxes
    geom_rect(aes(xmin = 0.7, xmax = 1.3, ymin = 1.7, ymax = 2.3), 
              fill = "white", color = "black") +
    geom_rect(aes(xmin = 2.7, xmax = 3.3, ymin = 1.7, ymax = 2.3), 
              fill = "white", color = "black") +
    geom_rect(aes(xmin = 1.7, xmax = 2.3, ymin = 2.7, ymax = 3.3), 
              fill = "white", color = "black") +
    
    # Add labels
    geom_text(aes(x = 1, y = 2, label = 'UPF consumption'), size = 3) +
    geom_text(aes(x = 3, y = 2, label = result$Region), size = 3) +
    geom_text(aes(x = 2, y = 3, label = result$Mediator), size = 3) +
    
    # Add effect estimates and CIs
    geom_text(aes(x = 2, y = 1.7, 
                  label = sprintf("Direct Effect: %.3f\n[%.3f, %.3f]\np = %.3f",
                                result$`ADE est`, 
                                result$`ADE CI low`, 
                                result$`ADE CI upp`,
                                result$`ADE p-value`)), size = 2.5) +
    geom_text(aes(x = 2, y = 3.5, 
                  label = sprintf("Indirect Effect: %.3f\n[%.3f, %.3f]\np = %.3f",
                                result$`ACME est`, 
                                result$`ACME CI low`, 
                                result$`ACME CI upp`,
                                result$`ACME p-value`)), size = 2.5) +
    
    # Add total effect
    geom_text(aes(x = 3.5, y = 2,
                  label = sprintf("Total Effect: %.3f\n[%.3f, %.3f]\np = %.3f",
                                result$`Total est`,
                                result$`Total CI low`,
                                result$`Total CI upp`,
                                result$`Total p-value`)), size = 2.5) +
    
    # Theme customization
    theme_void() +
    theme(plot.margin = margin(1, 1, 1, 1, "cm")) +
    coord_cartesian(xlim = c(0.5, 4), ylim = c(1.5, 4))
}

In [ ]:
library(openxlsx)
library(flextable)

save_flextables_to_xlsx <- function(flextable_list, output_path) {
  # Create a new workbook
  wb <- createWorkbook()
  
  # Counter for table numbering
  table_counter <- 1
  
  # Process each flextable in the list
  for(ft in flextable_list) {
    # Get the table title from the caption
    original_name <- ft$caption$value
    
    # Add sequential number after "Table S"
    sheet_name <- gsub("Table S ", paste0("Table S", table_counter, " "), original_name)
    
    # Update the caption in the flextable object
    ft$caption$value <- sheet_name
    
    # Clean up sheet name for Excel compatibility
    excel_sheet_name <- sheet_name
    # First remove asterisks specifically
    excel_sheet_name <- gsub("\\*", "", excel_sheet_name)
    # Then remove other invalid characters
    excel_sheet_name <- gsub("[/\\?:\\[\\]\\\\]", "", excel_sheet_name)
    excel_sheet_name <- gsub("^[0-9]", "T\\1", excel_sheet_name) # Ensure doesn't start with number
    excel_sheet_name <- gsub("'", "", excel_sheet_name) # Remove single quotes
    excel_sheet_name <- substr(excel_sheet_name, 1, 31) # Excel sheet names must be ≤ 31 chars
    
    # Print before/after for verification
    message(sprintf("Sheet %d cleaning: \nBefore: '%s' \nAfter:  '%s'", 
                   table_counter, sheet_name, excel_sheet_name))
    
    # Ensure unique sheet name
    base_name <- excel_sheet_name
    suffix <- 1
    while(base_name %in% names(wb)) {
      base_name <- substr(paste0(excel_sheet_name, "_", suffix), 1, 31)
      suffix <- suffix + 1
    }
    excel_sheet_name <- base_name
    
    # Add a new worksheet with error checking
    tryCatch({
      addWorksheet(wb, excel_sheet_name)
      
      # Extract the data from the flextable
      table_data <- ft$body$dataset
      
      # Write the title as the first row
      writeData(wb, excel_sheet_name, sheet_name, startRow = 1)
      
      # Write the data to the worksheet, starting from row 2
      writeData(wb, excel_sheet_name, table_data, startRow = 2)
      
      # Formatting
      headerStyle <- createStyle(textDecoration = "bold")
      titleStyle <- createStyle(textDecoration = "bold", fontSize = 12)
      
      # Apply styles
      addStyle(wb, excel_sheet_name, titleStyle, rows = 1, cols = 1)
      addStyle(wb, excel_sheet_name, headerStyle, rows = 2, cols = 1:ncol(table_data))
      
      # Auto-adjust column widths
      setColWidths(wb, excel_sheet_name, cols = 1:ncol(table_data), widths = "auto")
      
    }, error = function(e) {
      warning(paste("Error processing sheet:", sheet_name, "\nError message:", e$message))
    })
    
    # Increment counter
    table_counter <- table_counter + 1
  }
  
  # Verify that all sheets were created
  if(length(names(wb)) != length(flextable_list)) {
    warning(sprintf("Expected %d sheets but created %d sheets", 
                   length(flextable_list), length(names(wb))))
  }
  
  # Save the workbook with error checking
  tryCatch({
    saveWorkbook(wb, output_path, overwrite = TRUE)
    message(sprintf("Successfully saved workbook to %s", output_path))
  }, error = function(e) {
    stop(paste("Error saving workbook:", e$message))
  })
  
  # Return the modified flextable list
  return(flextable_list)
}


In [ ]:
get_r2_values <- function(full_model) {
    # Get response variable from full model
    response <- model.response(model.frame(full_model))
    
    # Get Percentage_NOVA_4 from model frame
    nova4 <- model.frame(full_model)$`data$Percentage_NOVA_4`
    
    # Simple model with just Percentage_NOVA_4
    simple_model <- lm(response ~ nova4)
    r2_without_covars <- summary(simple_model)$r.squared
    
    # Calculate partial R² 
    model_terms <- terms(full_model)
    covars_formula <- update(formula(full_model), . ~ . - data$Percentage_NOVA_4)
    covar_model <- update(full_model, covars_formula)
    
    ss_residual_covars <- sum(resid(covar_model)^2)
    ss_residual_full <- sum(resid(full_model)^2)
    partial_r2 <- (ss_residual_covars - ss_residual_full) / ss_residual_covars
    
    return(c(r2_without_covars, partial_r2))
}

get_aic_values <- function(full_model) {
    # Get response variable from full model
    response <- model.response(model.frame(full_model))
    
    # Get Percentage_NOVA_4 from model frame
    nova4 <- model.frame(full_model)$`data$Percentage_NOVA_4`
    
    # Simple model with just Percentage_NOVA_4
    simple_model <- lm(response ~ nova4)
    aic_simple <- AIC(simple_model)
    
    # Get AIC for full model
    aic_full <- AIC(full_model)
    
    # Return both AIC values
    return(c(aic_simple, aic_full))
}

In [ ]:
# Custom function to round and format values
round_and_format <- function(x, digits = 4, threshold = 0.0001) {
  # Round the values first
  x_rounded <- round(x, digits)
  
  # Format values in scientific notation if they are below the threshold and non-zero
  x_formatted <- ifelse(abs(x) < threshold & abs(x) > 0, format(x, scientific = TRUE, digits = digits), x_rounded)
  
  return(x_formatted)
}


In [ ]:
plot_struct=c('caudal anterior cingulate','caudal middle frontal','cuneus','entorhinal','fusiform','inferior parietal',
 'inferior temporal','isthmus cingulate','lateral occipital','lateral orbitofrontal','lingual',
 'medial orbitofrontal','middle temporal','parahippocampal','paracentral','pars opercularis','pars orbitalis',
 'pars triangularis','pericalcarine','postcentral','posterior cingulate','precentral','precuneus',
 'rostral anterior cingulate','rostral middle frontal','superior frontal','superior parietal','superior temporal',
 'supramarginal', 'transverse temporal','insula')

re_from <- "\\b([[:alpha:]])([[:alpha:]]+)"

plot_struct_title=gsub(re_from, "\\U\\1\\L\\2" ,plot_struct, perl=TRUE)

WM_names=c("Middle Cerebellar Peduncle",
"Pontine Crossing Tract",
"Genu of Corpus Callosum",
"Body of Corpus Callosum",
"Splenium of Corpus Callosum",
"Fornix",
"Corticospinal Tract Right",
"Corticospinal Tract Left",
"Medial Lemniscus Right",
"Medial Lemniscus Left",
"Inferior Cerebellar Peduncle Right",
"Inferior Cerebellar Peduncle Left",
"Superior Cerebellar Peduncle Right",
"Superior Cerebellar Peduncle Left",
"Cerebral Peduncle Right",
"Cerebral Peduncle Left",
"Anterior Limb of Internal Capsule Right",
"Anterior Limb of Internal Capsule Left",
"Posterior Limb of Internal Capsule Right",
"Posterior Limb of Internal Capsule Left",
"Retrolenticular Part of Internal Capsule Right",
"Retrolenticular Part of Internal Capsule Left",
"Anterior Corona Radiata Right",
"Anterior Corona Radiata Left",
"Superior Corona Radiata Right",
"Superior Corona Radiata Left",
"Posterior Corona Radiata Right",
"Posterior Corona Radiata Left",
"Posterior Thalamic Radiation Right",
"Posterior Thalamic Radiation Left",
"Sagittal Stratum Right",
"Sagittal Stratum Left",
"External Capsule Right",
"External Capsule Left",
"Cingulum Cingulate Gyrus Right",
"Cingulum Cingulate Gyrus Left",
"Cingulum Hippocampus Right",
"Cingulum Hippocampus Left",
"Fornix Cres Stria Terminalis Right",
"Fornix Cres Stria Terminalis Left",
"Superior Longitudinal Fasciculus Right",
"Superior Longitudinal Fasciculus Left",
"Superior Fronto-Occipital Fasciculus Right",
"Superior Fronto-Occipital Fasciculus Left",
"Uncinate Fasciculus Right",
"Uncinate Fasciculus Left",
"Tapetum Right",
"Tapetum Left")

# Read in the data

In [ ]:
data=read.table('/dagher/dagher11/filip/UPF/data/final_dataframe_all.csv',quote='"', sep=',', header=T)

In [ ]:
data_novanew=read.table('/dagher/dagher11/filip/UPF/data/NOVA_energy_averaged.csv',quote='"', sep=',', header=T)

In [ ]:
data=merge(data,data_novanew, all.x=T, by.x='eid', by.y='participant')

In [ ]:
data=data[!(is.na(data$Percentage_NOVA_4)),]

In [ ]:
nrow(data)

In [ ]:
data <- data[data$Total_Energy_KJ >= 2092 & data$Total_Energy_KJ <= 20920, ]

In [ ]:
nrow(data)

# Calculate VIF for the covariates

In [ ]:
m1=(lm(data$`mean_thickness_of_cuneus_left_hemisphere_27176.2.0` ~ 
          data$Percentage_NOVA_4 +
          data$saturated_fa +
          data$total_sugar + 
          data$sodium +
          data$body_mass_index_bmi_21001.2.0 +
          data$Total_Energy_KJ +
          data$age_when_attended_assessment_centre_21003.2.0+
          data$sex_31.0.0 + 
          (difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                   min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days')) +
          data$uk_biobank_assessment_centre_54.2.0 +
          data$qualifications_6138.2.0  +
          data$smoking_status_20116.2.0 +
          data$alcohol + 
          data$average_total_household_income_before_tax_738.2.0 +
          data$number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0 +
          data$volume_of_estimatedtotalintracranial_whole_brain_26521.2.0 
                ))

In [ ]:
car::vif(m1)
confint(m1)

# Sample characteristics

In [ ]:
psych::describe(data$age_when_attended_assessment_centre_21003.2.0)
psych::describe(data$body_mass_index_bmi_21001.2.0)
table(data$sex_31.0.0)
psych::describe(data$upf_perc)
psych::describe(data$kJ_sum)

In [ ]:
Sample_ukbbrain=data[!(is.na(data$mean_thickness_of_caudalanteriorcingulate_right_hemisphere_27267.2.0)),]
Sample_GMdiff=data[!(is.na(data$left_nucleus_accumbens_MD)),]
Sample_GMcell=data[!(is.na(data$MNI_PD25_subcortical_LH_nucleus_accumbens.nii.gz.ICVF)),]
Sample_cog1=data[!(is.na(data$number_of_incorrect_matches_in_round_399.2.1)),]
Sample_cog2=data[!(is.na(data$number_of_puzzles_correct_21004.2.0)),]
Sample_blood=data[!(is.na(data$c.reactive_protein_30710.0.0)),]

In [ ]:
nrow(Sample_GMdiff)

In [ ]:
psych::describe(Sample_ukbbrain$age_when_attended_assessment_centre_21003.2.0)
psych::describe(Sample_ukbbrain$body_mass_index_bmi_21001.2.0)
table(Sample_ukbbrain$sex_31.0.0)

In [ ]:
psych::describe(Sample_GMdiff$age_when_attended_assessment_centre_21003.2.0)
psych::describe(Sample_GMdiff$body_mass_index_bmi_21001.2.0)
table(Sample_GMdiff$sex_31.0.0)

In [ ]:
psych::describe(Sample_GMcell$age_when_attended_assessment_centre_21003.2.0)
psych::describe(Sample_GMcell$body_mass_index_bmi_21001.2.0)
table(Sample_GMcell$sex_31.0.0)

In [ ]:
psych::describe(Sample_cog1$age_when_attended_assessment_centre_21003.2.0)
psych::describe(Sample_cog1$body_mass_index_bmi_21001.2.0)
table(Sample_cog1$sex_31.0.0)

In [ ]:
psych::describe(Sample_cog2$age_when_attended_assessment_centre_21003.2.0)
psych::describe(Sample_cog2$body_mass_index_bmi_21001.2.0)
table(Sample_cog2$sex_31.0.0)

In [ ]:
psych::describe(Sample_blood$age_when_attended_assessment_centre_21003.2.0)
psych::describe(Sample_blood$body_mass_index_bmi_21001.2.0)
table(Sample_blood$sex_31.0.0)

# Metabolic analyses

In [ ]:
corrs=c('c.reactive_protein_30710.0.0','triglycerides_30870.0.0','glycated_haemoglobin_hba1c_30750.0.0', 
         'hdl_cholesterol_30760.0.0', 'ldl_direct_30780.0.0',
         'systolic_blood_pressure_automated_reading_4080.0.0', 'diastolic_blood_pressure_automated_reading_4079.0.0',
         'sodium','total_sugar','saturated_fa','visceral_adipose_tissue_volume_vat_22407.2.0', 'WHR', 'body_mass_index_bmi_21001.2.0')

In [ ]:
results=matrix(ncol=13,nrow=13)

for (i in 1:13) 
{ 
    
    m1=(lm(scale(data[[corrs[i]]]) ~ 
          data$Percentage_NOVA_4 +
          poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
          data$sex_31.0.0 + 
          data$qualifications_6138.2.0  +
          data$smoking_status_20116.2.0 +
          data$alcohol + 
          data$average_total_household_income_before_tax_738.2.0 +
          data$number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0 +
          data$number_of_daysweek_of_moderate_physical_activity_10._minutes_884.2.0 
                ))

    sum_mod=summary(m1)
    
    
    results[i,1]=corrs[i]
    results[i,2]=sum_mod$coefficients[2,4] # p-value 
    results[i,3]=sum_mod$coefficients[2,3] # t-value 
    results[i,4]=sum_mod$coefficients[2,1] # est for 
    results[i,5]=sum_mod$coefficients[2,2] # se for 
    results[i,6]=''
    results[i,7]=sum_mod$df[2]+sum_mod$df[1] #
    results[i,8]=confint(m1)[2,1]
    results[i,9]=confint(m1)[2,2]
    results[i,10] <- get_r2_values(m1)[1]  # R² without covariates
    results[i,11] <- get_r2_values(m1)[2] # Partial R² with covariates
    results[i,12] <- get_aic_values(m1)[1]  # AIC without covariates
    results[i,13] <- get_aic_values(m1)[2] # AIC with covariates

}


colnames(results)=c('Label','p-value', 't', 'est', 'se', 'hemisphere','n','CI_low','CI_up', 'R2_no','R2_yes', 'AIC_no','AIC_yes')
results[,2]=p.adjust(results[,2], method='BH')
results=as.data.frame(results)

results$t=as.numeric(results$t)
results$p=as.numeric(results$p)
results$se=as.numeric(results$se)
results$est=as.numeric(results$est)
results$CI_low=as.numeric(results$CI_low)
results$CI_up=as.numeric(results$CI_up)
results$R2_no=as.numeric(results$R2_no)
results$R2_yes=as.numeric(results$R2_yes)

In [ ]:
results$FDR <- ifelse(as.numeric(results$'p-value') < 0.05, "p < 0.05", "p >= 0.05")
results$Label=c('CRP', 'TG', 'HbA1c', 'HDL', 'LDL', 'Systolic blood pressure', 'Diastolic blood pressure', 
                'Sodium', 'Total dietary sugar', 'Saturated fatty acids', 'Visceral adipose tissue','WHR','BMI')
results$R2_no=as.numeric(results$R2_no)
results$R2_yes=as.numeric(results$R2_yes)
png('/dagher/dagher11/filip/UPF/figures/Base_correlations.png', width=1500, height=1000, res=300)
results %>%
  arrange(desc(as.numeric(est))) %>%    
  mutate(Label=factor(Label, levels=Label)) %>%   
  ggplot(aes(x=Label, y=as.numeric(est), ymin=as.numeric(CI_low), ymax=as.numeric(CI_up), colour=FDR)) +
        geom_pointrange() + 
        geom_hline(yintercept=0, lty=2) +  # add a dotted line at x=0 after flip
        coord_flip() +  # flip coordinates (puts labels on y axis)
        xlab("") + ylab("Beta/95% CI") +
        theme_bw() +# use a white background
        scale_colour_manual(values=c("p < 0.05" = "#ed563d", "p >= 0.05" = "#000000"))  # specify colours
dev.off()

In [ ]:
results=results %>% mutate_if(is.numeric, round_and_format, digits = 4, threshold = 0.0001)
tab_corr=flextable(results)
tab_corr=autofit(tab_corr, add_w = 0.1,add_h = 0.1,part = c("body", "header"),unit = "in")  %>% fit_to_width(max_width=8) %>% 
set_caption(tab_corr, caption = 'Table S1 Associations between ultraprocessed food intake, metabolic variables, and nutrients intake')

# Brain data

## Cortical thickness and surface area

In [ ]:
results=matrix(ncol=13,nrow=length(c(grep('area_of',colnames(data)),
                                   grep('mean_thickness_of',colnames(data))))) 

for (i in 1:length(c(grep('area_of',colnames(data)),
                    grep('mean_thickness_of',colnames(data))))) 
{ 
    
    m1=(lm(data[c(grep('area_of',colnames(data)),
                           grep('mean_thickness_of',colnames(data)))][[i]] ~ 
          data$Percentage_NOVA_4 +
          data$saturated_fa +
          data$total_sugar + 
          data$sodium +
          data$body_mass_index_bmi_21001.2.0 +
          data$Total_Energy_KJ +
          poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
          data$sex_31.0.0 + 
          poly(difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                   min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
          data$uk_biobank_assessment_centre_54.2.0 +
          data$qualifications_6138.2.0  +
          data$smoking_status_20116.2.0 +
          data$alcohol + 
          data$average_total_household_income_before_tax_738.2.0 +
          data$number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0 +
          data$volume_of_estimatedtotalintracranial_whole_brain_26521.2.0 
                ))

    sum_mod=summary(m1)
    
    results[i,1]=colnames(data)[c(grep('area_of',colnames(data)),
                                   grep('mean_thickness_of',colnames(data)))][i] 
    results[i,2]=sum_mod$coefficients[2,4] # p-value 
    results[i,3]=sum_mod$coefficients[2,3] # t-value 
    results[i,4]=sum_mod$coefficients[2,1] # est for 
    results[i,5]=sum_mod$coefficients[2,2] # se for 
    results[i,6]=ifelse(grepl("left",results[i,1]),"left","right") #save hemisphere
    results[i,7]=sum_mod$df[2]+sum_mod$df[1] #
    results[i,8]=confint(m1)[2,1]
    results[i,9]=confint(m1)[2,2]
    results[i,10] <- get_r2_values(m1)[1]  # R² without covariates
    results[i,11] <- get_r2_values(m1)[2] # Partial R² with covariates
    results[i,12] <- get_aic_values(m1)[1]  # AIC without covariates
    results[i,13] <- get_aic_values(m1)[2] # AIC with covariates

}


colnames(results)=c('Parcel','p-value', 't-value', 'est', 'se', 'hemisphere','n','CI_low','CI_up','R2_no','R2_yes','AIC_no','AIC_yes')

In [ ]:
results[1:62,2]=p.adjust(results[1:62,2], method='BH')
results[63:124,2]=p.adjust(results[63:124,2], method='BH')
results=as.data.frame(results)
results$measure=c(rep('Area', 62),rep('Thickness',62))
colnames(results)=c('region','p','t','est','se','hemi','Sample size','CI_low','CI_up','R2_no','R2_yes','AIC_no','AIC_yes','phenotype')
results$t=as.numeric(results$t)
results$p=as.numeric(results$p)
results$se=as.numeric(results$se)
results$est=as.numeric(results$est)
results$CI_low=as.numeric(results$CI_low)
results$CI_up=as.numeric(results$CI_up)
results$R2_no=as.numeric(results$R2_no)
results$R2_yes=as.numeric(results$R2_yes)
results$AIC_no=as.numeric(results$AIC_no)
results$AIC_yes=as.numeric(results$AIC_yes)
results$region=rep(plot_struct_title,4)

In [ ]:
plot_brain = results[results$p<0.05,] %>%
              group_by(phenotype) %>%
              ggplot() + geom_brain(atlas = dkt, 
                         #position = position_brain(hemi ~ side),
                         aes(fill = t, color=I("darkgrey"))) +              
                         facet_wrap(.~phenotype, nrow=2) +
                         #ggtitle(label) + 
                         theme_minimal() +
                         theme(axis.text.y = element_blank(),
                               axis.text.x = element_blank(),
                               strip.background =element_rect(fill="white", colour='white'),
                               strip.text = element_text(colour = 'black', family="Arial"),
                               plot.title = element_text(hjust = 0.5, family="Arial"),
                               legend.position="bottom",
                               panel.grid.major = element_blank(), 
                               panel.grid.minor = element_blank()) + scale_fill_gradient2(midpoint=0, low="#00203FFF", mid="white",
                                 high="#ADEFD1FF", space ="Lab", na.value='white')

In [ ]:
results$FDR <- ifelse(results$p < 0.05, "p < 0.05", "p >= 0.05")
results$hemi_offset <- ifelse(results$hemi == "left", 0.1, -0.1)

results <- results %>%
   mutate(legend_color = interaction(FDR, hemi))

results$legend_color=factor(results$legend_color, levels=c('p >= 0.05.left', 'p >= 0.05.right','p < 0.05.left', 'p < 0.05.right' ))

region_order <- results %>%
  arrange(desc(est)) %>%
  pull(region) %>%
  unique()

# Then use this order in your plot
forest_CTSA <- results %>%
  mutate(region = factor(region, levels = region_order)) %>%
ggplot(aes(x=region, y=est, ymin=as.numeric(CI_low), ymax=as.numeric(CI_up), colour=legend_color)) +
        geom_pointrange(position = position_nudge(x = results$hemi_offset)) + 
        geom_hline(yintercept=0, lty=2) +  
        coord_flip() +
        xlab("") + ylab("Beta/95% CI") +
        theme_bw() + 
        facet_grid(~phenotype, scales="free") + 
                scale_colour_manual(values=c("p >= 0.05.right" = "#F5C7B8FF", "p >= 0.05.left" = "#9FB1BCFF", 
                                     "p < 0.05.right" = "#e05022", "p < 0.05.left" = "#2E5266FF"),
                           labels = c("p>0.05 (Left)", 
                                  "p>0.05 (Right/BL)",
                                  "p<0.05 (Left)",
                                  "p<0.05 (Right/BL)"), drop=FALSE) +
        guides(colour = guide_legend(title = "")) +
        theme(legend.position="left", strip.background =element_rect(fill="white")) + 
        scale_y_continuous(n.breaks = 4)

In [ ]:
results=results %>% mutate_if(is.numeric, round_and_format, digits = 4, threshold = 0.0001)
tab_cortex=flextable(results)
tab_cortex=autofit(tab_cortex, add_w = 0.1,add_h = 0.1,part = c("body", "header"),unit = "in")  %>% fit_to_width(max_width=8) %>% 
set_caption(tab_cortex, caption = 'Table S2 Associations between cortical thickness and area and ultraprocessed food intake')

## Subcortical volumes

In [ ]:
results=matrix(ncol=13,nrow=length(grep('volume_of',colnames(data))[c(1:7, 103:107, 8:9)])) 

        for (i in 1:length(c(grep('volume_of',colnames(data))[c(1:7, 103:107, 8:9)]))) 
        { 
            
            m=lm(data[grep('volume_of',colnames(data))[c(1:7, 103:107, 8:9)]][[i]]/
                                   data$volume_of_estimatedtotalintracranial_whole_brain_26521.2.0 ~ 
                      data$Percentage_NOVA_4 +
                      data$saturated_fa +
                      data$total_sugar + 
                      data$sodium +
                      data$body_mass_index_bmi_21001.2.0 +
                      data$Total_Energy_KJ +
                      poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
                      data$sex_31.0.0 + 
                      poly(difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                               min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
                      data$uk_biobank_assessment_centre_54.2.0 +
                      data$qualifications_6138.2.0  +
                      data$smoking_status_20116.2.0 +
                      data$alcohol + 
                      data$average_total_household_income_before_tax_738.2.0 +
                      data$number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0
                        )

            sum_mod=summary(m)

                        results[i,1]=colnames(data)[grep('volume_of',colnames(data))[c(1:7, 103:107, 8:9)]][i] 
                        results[i,2]=sum_mod$coefficients[2,4] # p-value 
                        results[i,3]=sum_mod$coefficients[2,3] # t-value 
                        results[i,4]=sum_mod$coefficients[2,1] # est for 
                        results[i,5]=sum_mod$coefficients[2,2] # se for 
                        results[i,6]=ifelse(grepl("left",results[i,1]),"left","right") #save hemisphere
                        results[i,7]=sum_mod$df[2]+sum_mod$df[1] #
                        results[i,8]=confint(m)[2,1]
                        results[i,9]=confint(m)[2,2]
                        results[i,10] <- get_r2_values(m)[1]  # R² without covariates
                        results[i,11] <- get_r2_values(m)[2] # Partial R² with covariates
                        results[i,12] <- get_aic_values(m)[1]  # AIC without covariates
                        results[i,13] <- get_aic_values(m)[2] # AIC with covariates

}


In [ ]:
results[,2]=p.adjust(results[,2], method='BH')
results=as.data.frame(results)
colnames(results)=c('label','p','t','est','se','hemisphere','Sample size','CI_low','CI_up','R2_no','R2_yes','AIC_no','AIC_yes')
results$t=as.numeric(results$t)
results$p=as.numeric(results$p)
results$est=as.numeric(results$est)
results$se=as.numeric(results$se)
results$CI_low=as.numeric(results$CI_low)
results$CI_up=as.numeric(results$CI_up)
results$R2_no=as.numeric(results$R2_no)
results$R2_yes=as.numeric(results$R2_yes)
results$AIC_no=as.numeric(results$AIC_no)
results$AIC_yes=as.numeric(results$AIC_yes)
results$label=c('Caudate','Caudate',
                    'Putamen','Putamen','Pallidum','Pallidum',
                    'Thalamus Proper','Thalamus Proper',
                    'Hippocampus','Hippocampus','Accumbens area','Accumbens area',
                    'Amygdala','Amygdala')

In [ ]:
   sub_plot = results[results$p<0.05,] %>% 
                  ggplot() + geom_brain(atlas = aseg, 
                             #position = position_brain(hemi ~ side),
                             aes(fill = t, color=I("black")), side = "coronal") +              
                             #ggtitle(paste(label, '', sep='')) + 
                             theme_minimal() +
                             theme(axis.text.y = element_blank(),
                                   axis.text.x = element_blank(),
                                   strip.background =element_rect(fill="white", colour='white'),
                                   strip.text = element_text(colour = 'black', family="Arial"),
                                   plot.title = element_text(hjust = 0.5, family="Arial"),
                                   legend.position="bottom",
                                   panel.grid.major = element_blank(), 
                                   panel.grid.minor = element_blank()) + 
                             scale_fill_gradient2(midpoint=0, low="#00203FFF", mid="white",
                                 high="#68e2aa", space ="Lab", na.value='white')

In [ ]:
results$FDR <- ifelse(results$p < 0.05, "p < 0.05", "p >= 0.05")
results_subvol=results
results_subvol$hemi_offset <- ifelse(results_subvol$hemisphere == "left", 0.1, -0.1)

region_order <- results %>%
  arrange(desc(est)) %>%
  pull(label) %>%
  unique()

forest_subvol=results_subvol %>%
  mutate(label = factor(label, levels = region_order)) %>%
ggplot(aes(x=label, y=est, ymin=as.numeric(CI_low), ymax=as.numeric(CI_up), colour=interaction(FDR,hemisphere))) +
        geom_pointrange(position = position_nudge(x = results_subvol$hemi_offset)) + 
        geom_hline(yintercept=0, lty=2) +  
        coord_flip() +
        xlab("") + ylab("Beta/95% CI") +
        theme_bw() + 
        scale_colour_manual(values=c("p >= 0.05.right" = "#F5C7B8FF", "p >= 0.05.left" = "#9FB1BCFF", 
                                     "p < 0.05.right" = "#e05022", "p < 0.05.left" = "#2E5266FF"),
                           labels = c("p>0.05 (Left)", 
                                  "p>0.05 (Right)",
                                  "p<0.05 (Left)",
                                  "p<0.05 (Right)")) +
        guides(colour = guide_legend(title = "")) +
        theme(legend.position="none", strip.background =element_rect(fill="white")) + 
        scale_y_continuous(n.breaks = 5)

In [ ]:
results=results %>% mutate_if(is.numeric, round_and_format, digits = 4, threshold = 0.0001)
tab_subcortex=flextable(results)
tab_subcortex=autofit(tab_subcortex, add_w = 0.1,add_h = 0.1,part = c("body", "header"),unit = "in")  %>% fit_to_width(max_width=8) %>% 
set_caption(caption = 'Table S3 Associations between subcortical volumes and ultraprocessed food intake')

## White matter measures

In [ ]:
results=matrix(ncol=13,nrow=length(c(grep('mean_fa',colnames(data)),
                                   grep('mean_md',colnames(data))))) 

for (i in 1:length(c(grep('mean_fa',colnames(data)),
                    grep('mean_md',colnames(data))))) 
{ 
    
    m1=(lm(data[c(grep('mean_fa',colnames(data)),
                           grep('mean_md',colnames(data)))][[i]] ~ 
          data$Percentage_NOVA_4 +
          data$saturated_fa +
          data$total_sugar + 
          data$sodium +
          data$body_mass_index_bmi_21001.2.0 +
          data$Total_Energy_KJ +
          poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
          data$sex_31.0.0 + 
          poly(difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                   min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
          data$uk_biobank_assessment_centre_54.2.0 +
          data$qualifications_6138.2.0  +
          data$smoking_status_20116.2.0 +
          data$alcohol + 
          data$average_total_household_income_before_tax_738.2.0 +
          data$number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0
))

    sum_mod=summary(m1)
    
    results[i,1]=colnames(data)[c(grep('mean_fa',colnames(data)),
                                   grep('mean_md',colnames(data)))][i] 
    results[i,2]=sum_mod$coefficients[2,4] # p-value 
    results[i,3]=sum_mod$coefficients[2,3] # t-value 
    results[i,4]=sum_mod$coefficients[2,1] # est for 
    results[i,5]=sum_mod$coefficients[2,2] # sd for 
    results[i,6]=ifelse(grepl("left", results[i, 1]), 
                        "left", 
                        ifelse(grepl("right", results[i, 1]), 
                               "right", 
                               ""))
    results[i,7]=sum_mod$df[2]+sum_mod$df[1] #
    results[i,8]=confint(m1)[2,1]
    results[i,9]=confint(m1)[2,2]
    results[i,10] <- get_r2_values(m1)[1]  # R² without covariates
    results[i,11] <- get_r2_values(m1)[2] # Partial R² with covariates
    results[i,12] <- get_aic_values(m1)[1]  # AIC without covariates
    results[i,13] <- get_aic_values(m1)[2] # AIC with covariates

}


colnames(results)=c('Parcel','p-value', 't-value', 'est', 'se', 'hemisphere','n','CI_low','CI_up','R2_no','R2_yes','AIC_no','AIC_yes')

In [ ]:
wm_names_plot=gsub(' Left','',WM_names)
wm_names_plot=gsub(' Right','',wm_names_plot)
results[1:48,2]=p.adjust(results[1:48,2], method='BH')
results[49:96,2]=p.adjust(results[49:96,2], method='BH')
results=as.data.frame(results)
results$measure=c(rep('FA', 48),rep('MD',48))
colnames(results)=c('region','p','t','est','se','hemi','Sample size','CI_low','CI_up','R2_no','R2_yes','AIC_no','AIC_yes','phenotype')
results$t=as.numeric(results$t)
results$p=as.numeric(results$p)
results$se=as.numeric(results$se)
results$est=as.numeric(results$est)
results$CI_low=as.numeric(results$CI_low)
results$CI_up=as.numeric(results$CI_up)
results$R2_no=as.numeric(results$R2_no)
results$R2_yes=as.numeric(results$R2_yes)
results$AIC_no=as.numeric(results$AIC_no)
results$AIC_yes=as.numeric(results$AIC_yes)
results$region=rep(wm_names_plot,2)

In [ ]:
results$FDR <- ifelse(results$p < 0.05, "p < 0.05", "p >= 0.05")
results_wm=results
results_wm$hemi_offset <- ifelse(results_wm$hemi == "left", 0.1, -0.1)
results_wm <- results_wm %>%
  mutate(hemi = ifelse(hemi == "", "right", hemi))

region_order <- results %>%
  arrange(desc(est)) %>%
  pull(region) %>%
  unique()

forest_WM=results_wm %>%
  mutate(region = factor(region, levels = region_order)) %>%
ggplot(aes(x=region, y=est, ymin=as.numeric(CI_low), ymax=as.numeric(CI_up), colour=interaction(FDR,hemi))) +
        geom_pointrange(position = position_nudge(x = results_wm$hemi_offset)) + 
        geom_hline(yintercept=0, lty=2) +  
        coord_flip() +
        xlab("") + ylab("Beta/95% CI") +
        theme_bw() + 
        facet_grid(~phenotype, scales='free')+
        scale_colour_manual(values=c("p >= 0.05.right" = "#F5C7B8FF", "p >= 0.05.left" = "#9FB1BCFF", 
                                     "p < 0.05.right" = "#e05022", "p < 0.05.left" = "#2E5266FF"),
                           labels = c("p>0.05 (Left)", 
                                  "p>0.05 (Right)",
                                  "p<0.05 (Left)",
                                  "p<0.05 (Right)")) +
        guides(colour = guide_legend(title = "")) +
        theme(legend.position="none", strip.background =element_rect(fill="white"))+ 
        scale_y_continuous(n.breaks = 4)

In [ ]:
results=results %>% mutate_if(is.numeric, round_and_format, digits = 4, threshold = 0.0001)
tab_wm=flextable(results)
tab_wm=autofit(tab_wm, add_w = 0.1,add_h = 0.1,part = c("body", "header"),unit = "in")  %>% fit_to_width(max_width=8) %>% 
set_caption(caption = 'Table S4 Associations between white matter diffusivity measures and ultraprocessed food intake')

## Subcortical DTI


In [ ]:
structures = c('left_caudate',
               'right_caudate',
               'left_putamen',
               'right_putamen',
               'left_globus_pallidus_externa',
               'right_globus_pallidus_externa',
               'left_thalamus',
               'right_thalamus',
               'left_hippocampus',
               'right_hippocampus',
               'left_nucleus_accumbens',
               'right_nucleus_accumbens',
               'left_amygdala',
               'right_amygdala',
               'full_hypo')

suffixes <- c("\\.FA", "\\.MD")#, "\\.RD", "\\.AD")

structures <- paste0(rep(structures, 2), rep(suffixes, each = length(structures)))

In [ ]:
results=matrix(ncol=13,nrow=length(structures))

for (i in 1:(length(structures))) 
{ 
    
    m1=(lm(data[[c(grep(structures[i],colnames(data)))]] ~ 
          data$Percentage_NOVA_4 +
          data$saturated_fa +
          data$total_sugar + 
          data$sodium +
          data$body_mass_index_bmi_21001.2.0 +
          data$Total_Energy_KJ +
          poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) +
          data$sex_31.0.0 + 
          poly(difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                   min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
          data$uk_biobank_assessment_centre_54.2.0 + 
          data$qualifications_6138.2.0  +
          data$smoking_status_20116.2.0 +
          data$alcohol + 
          data$average_total_household_income_before_tax_738.2.0 +
          data$number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0 
                 ))

    sum_mod=summary(m1)
    
    results[i,1]=structures[i]
    results[i,2]=sum_mod$coefficients[2,4] # p-value 
    results[i,3]=sum_mod$coefficients[2,3] # t-value 
    results[i,4]=sum_mod$coefficients[2,1] # est for 
    results[i,5]=sum_mod$coefficients[2,2] # se for 
    results[i,6]=ifelse(grepl("left", results[i, 1]), 
                        "left", 
                        ifelse(grepl("right", results[i, 1]), 
                               "right", 
                               ""))
    results[i,7]=sum_mod$df[2]+sum_mod$df[1] #
    results[i,8]=confint(m1)[2,1]
    results[i,9]=confint(m1)[2,2]
    results[i,10] <- get_r2_values(m1)[1]  # R² without covariates
    results[i,11] <- get_r2_values(m1)[2] # Partial R² with covariates
    results[i,12] <- get_aic_values(m1)[1]  # AIC without covariates
    results[i,13] <- get_aic_values(m1)[2] # AIC with covariates

}


colnames(results)=c('Parcel','p-value', 't-value', 'est', 'se', 'hemisphere','n','CI_low','CI_up','R2_no','R2_yes','AIC_no','AIC_yes')


In [ ]:
results[1:15,2]=p.adjust(results[1:15,2], method='BH')
results[16:30,2]=p.adjust(results[16:30,2], method='BH')
#results[31:45,2]=p.adjust(results[31:45,2], method='BH')
#results[46:60,2]=p.adjust(results[46:60,2], method='BH')
results=as.data.frame(results)
results$measure=c(rep('FA', 15),rep('MD',15))#,rep('RD',15),rep('AD',15))
colnames(results)=c('label','p','t','est','se','hemi','Sample size','CI_low','CI_up','R2_no','R2_yes','AIC_no','AIC_yes','phenotype')
results$t=as.numeric(results$t)
results$p=as.numeric(results$p)
results$se=as.numeric(results$se)
results$est=as.numeric(results$est)
results$CI_low=as.numeric(results$CI_low)
results$CI_up=as.numeric(results$CI_up)
results$R2_no=as.numeric(results$R2_no)
results$R2_yes=as.numeric(results$R2_yes)
results$AIC_no=as.numeric(results$AIC_no)
results$AIC_yes=as.numeric(results$AIC_yes)
results$label=rep(c('Caudate','Caudate',
                    'Putamen','Putamen','Pallidum','Pallidum',
                    'Thalamus Proper','Thalamus Proper',
                    'Hippocampus','Hippocampus','Accumbens area','Accumbens area',
                    'Amygdala','Amygdala','Hypothalamus'),2)
#results$region=NULL

In [ ]:
results$label=rep(c('Left-Caudate','Right-Caudate',
                    'Left-Putamen','Right-Putamen','Left-Pallidum','Right-Pallidum',
                    'Left-Thalamus-Proper','Right-Thalamus-Proper',
                    'Left-Hippocampus','Right-Hippocampus','Left-Accumbens-area','Right-Accumbens-area',
                    'Left-Amygdala','Right-Amygdala','Hypothalamus'),2)

tiff('/dagher/dagher11/filip/UPF/figures/DTI_plot.tif', width=1600, height=2000, res=300)
results %>% group_by(phenotype) %>%
                  ggplot() + geom_brain(atlas = aseg, 
                             #position = position_brain(hemi ~ side),
                             aes(fill = t, color=I("black")), side = "coronal") +              
                             #ggtitle(paste(label, '', sep='')) +  +              
                             facet_wrap(.~phenotype, nrow=2) +
                             theme_minimal() +
                             theme(axis.text.y = element_blank(),
                                   axis.text.x = element_blank(),
                                   strip.background =element_rect(fill="white", colour='white'),
                                   strip.text = element_text(colour = 'black', family="Arial"),
                                   plot.title = element_text(hjust = 0.5, family="Arial"),
                                   legend.position="bottom",
                                   panel.grid.major = element_blank(), 
                                   panel.grid.minor = element_blank()) + 
                             scale_fill_gradient2(midpoint=0, low="#00203FFF", mid="white",
                                 high="#68e2aa", space ="Lab", na.value='white')
dev.off()

In [ ]:
results$FDR <- ifelse(results$p < 0.05, "p < 0.05", "p >= 0.05")
results$hemi_offset <- ifelse(results$hemi == "left", 0.1, -0.1)
results_subDTI=results
results_subDTI <- results_subDTI %>%
  mutate(hemi = ifelse(hemi == "", "right", hemi))
results_subDTI <- results_subDTI %>%
   mutate(legend_color = interaction(FDR, hemi))

results_subDTI$legend_color=factor(results_subDTI$legend_color, levels=c('p >= 0.05.left', 'p >= 0.05.right','p < 0.05.left', 'p < 0.05.right' ))

region_order <- results %>%
  arrange(desc(est)) %>%
  pull(label) %>%
  unique()

forest_subDTI=results_subDTI %>%
  mutate(label = factor(label, levels = region_order)) %>% 
ggplot(aes(x=label, y=est, ymin=as.numeric(CI_low), ymax=as.numeric(CI_up), colour=legend_color)) +
        geom_pointrange(position = position_nudge(x = results$hemi_offset)) + 
        geom_hline(yintercept=0, lty=2) +  
        coord_flip() +
        xlab("") + ylab("Beta/95% CI") +
        theme_bw() + 
        facet_grid(~phenotype, scales='free') +
        scale_colour_manual(values=c("p >= 0.05.right" = "#F5C7B8FF", "p >= 0.05.left" = "#9FB1BCFF", 
                                     "p < 0.05.right" = "#e05022", "p < 0.05.left" = "#2E5266FF"),
                           labels = c("p>0.05 (Left)", 
                                  "p>0.05 (Right/BL)",
                                  "p<0.05 (Left)",
                                  "p<0.05 (Right/BL)"), drop=FALSE) +
        guides(colour = guide_legend(title = "")) +
        theme(legend.position="left", strip.background =element_rect(fill="white"),
              plot.margin = margin(t = 5, r = 20, b = 5, l = 5, unit = "pt")  )+ 
        scale_y_continuous(n.breaks = 4)

In [ ]:
results=results %>% mutate_if(is.numeric, round_and_format, digits = 4, threshold = 0.0001)
tab_dti=flextable(results)
tab_dti=autofit(tab_dti, add_w = 0.1,add_h = 0.1,part = c("body", "header"),unit = "in")  %>% fit_to_width(max_width=8) %>% 
set_caption(caption = 'Table S5 Associations between subcortical grey matter diffusivity measures and ultraprocessed food intake')

## NODDI measures

In [ ]:
structures = c('MNI_PD25_subcortical_LH_caudate.nii.gz',
               'MNI_PD25_subcortical_RH_caudate.nii.gz',
               'MNI_PD25_subcortical_LH_putamen.nii.gz',
               'MNI_PD25_subcortical_RH_putamen.nii.gz',
               'globus_pallidus_LH',
               'globus_pallidus_RH',
               'MNI_PD25_subcortical_LH_thalamus.nii.gz',
               'MNI_PD25_subcortical_RH_thalamus.nii.gz',
               'MNI_PD25_subcortical_LH_hippocampus.nii.gz',
               'MNI_PD25_subcortical_RH_hippocampus.nii.gz',
               'MNI_PD25_subcortical_LH_nucleus_accumbens.nii.gz',
               'MNI_PD25_subcortical_RH_nucleus_accumbens.nii.gz',
               'MNI_PD25_subcortical_LH_amygdala.nii.gz',
               'MNI_PD25_subcortical_RH_amygdala.nii.gz',
               'full_hypo')

suffixes <- c("\\.ICVF", "\\.ISOVF")

structures <- paste0(rep(structures, 2), rep(suffixes, each = length(structures)))

In [ ]:
results=matrix(ncol=13,nrow=length(structures))

for (i in 1:(length(structures))) 
{ 
    
    m1=(lm(data[[c(grep(structures[i],colnames(data)))]] ~ 
          data$Percentage_NOVA_4 +
          data$saturated_fa +
          data$total_sugar + 
          data$sodium +
          data$body_mass_index_bmi_21001.2.0 +
          data$Total_Energy_KJ +
          poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
          data$sex_31.0.0 + 
          poly(difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                   min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
          data$uk_biobank_assessment_centre_54.2.0 + 
          data$qualifications_6138.2.0  +
          data$smoking_status_20116.2.0 +
          data$alcohol + 
          data$average_total_household_income_before_tax_738.2.0 +
          data$number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0 
                 ))

    sum_mod=summary(m1)
    
    results[i,1]=structures[i]
    results[i,2]=sum_mod$coefficients[2,4] # p-value 
    results[i,3]=sum_mod$coefficients[2,3] # t-value 
    results[i,4]=sum_mod$coefficients[2,1] # est for 
    results[i,5]=sum_mod$coefficients[2,2] # se for 
    results[i,6]=ifelse(grepl("LH", results[i, 1]), 
                        "left", 
                        ifelse(grepl("RH", results[i, 1]), 
                               "right", 
                               ""))
    results[i,7]=sum_mod$df[2]+sum_mod$df[1] #
    results[i,8]=confint(m1)[2,1]
    results[i,9]=confint(m1)[2,2]
    results[i,10] <- get_r2_values(m1)[1]  # R² without covariates
    results[i,11] <- get_r2_values(m1)[2] # Partial R² with covariates
    results[i,12] <- get_aic_values(m1)[1]  # AIC without covariates
    results[i,13] <- get_aic_values(m1)[2] # AIC with covariates

}


colnames(results)=c('Parcel','p-value', 't-value', 'est', 'se', 'hemisphere','n','CI_low','CI_up','R2_no','R2_yes','AIC_no','AIC_yes')

In [ ]:
results[1:15,2]=p.adjust(results[1:15,2], method='BH')
results[16:30,2]=p.adjust(results[16:30,2], method='BH')
#results[31:45,2]=p.adjust(results[31:45,2], method='BH')
#results[52:68,2]=p.adjust(results[52:68,2], method='BH')
results=as.data.frame(results)
results$measure=c(rep('ICVF', 15),rep('ISOVF',15))
colnames(results)=c('label','p','t','est','se','hemi','Sample size','CI_low','CI_up','R2_no','R2_yes','AIC_no','AIC_yes','phenotype')
results$t=as.numeric(results$t)
results$p=as.numeric(results$p)
results$se=as.numeric(results$se)
results$est=as.numeric(results$est)
results$CI_low=as.numeric(results$CI_low)
results$CI_up=as.numeric(results$CI_up)
results$R2_no=as.numeric(results$R2_no)
results$R2_yes=as.numeric(results$R2_yes)
results$AIC_no=as.numeric(results$AIC_no)
results$AIC_yes=as.numeric(results$AIC_yes)
results$label=rep(c('Caudate','Caudate',
                    'Putamen','Putamen','Pallidum','Pallidum',
                    'Thalamus Proper','Thalamus Proper',
                    'Hippocampus','Hippocampus','Accumbens area','Accumbens area',
                    'Amygdala','Amygdala','Hypothalamus'),2)

In [ ]:
results$label=rep(c('Left-Caudate','Right-Caudate',
                    'Left-Putamen','Right-Putamen','Left-Pallidum','Right-Pallidum',
                    'Left-Thalamus-Proper','Right-Thalamus-Proper',
                    'Left-Hippocampus','Right-Hippocampus','Left-Accumbens-area','Right-Accumbens-area',
                    'Left-Amygdala','Right-Amygdala','Hypothalamus'),2)

tiff('/dagher/dagher11/filip/UPF/figures/NODDI_plot.tif', width=1600, height=2000, res=300)
results %>% group_by(phenotype) %>%
                  ggplot() + geom_brain(atlas = aseg, 
                             #position = position_brain(hemi ~ side),
                             aes(fill = t, color=I("black")), side = "coronal") +              
                             #ggtitle(paste(label, '', sep='')) +  +              
                             facet_wrap(.~phenotype, nrow=2) +
                             theme_minimal() +
                             theme(axis.text.y = element_blank(),
                                   axis.text.x = element_blank(),
                                   strip.background =element_rect(fill="white", colour='white'),
                                   strip.text = element_text(colour = 'black', family="Arial"),
                                   plot.title = element_text(hjust = 0.5, family="Arial"),
                                   legend.position="bottom",
                                   panel.grid.major = element_blank(), 
                                   panel.grid.minor = element_blank()) + 
                             scale_fill_gradient2(midpoint=0, low="#00203FFF", mid="white",
                                 high="#68e2aa", space ="Lab", na.value='white')
dev.off()

In [ ]:
results$FDR <- ifelse(results$p < 0.05, "p < 0.05", "p >= 0.05")
results$hemi_offset <- ifelse(results$hemi == "left", 0.1, -0.1)
results_subNODDI=results
results_subNODDI <- results_subNODDI %>%
  mutate(hemi = ifelse(hemi == "", "right", hemi))

region_order <- results %>%
  arrange(desc(est)) %>%
  pull(label) %>%
  unique()

forest_subNODDI=results_subNODDI %>%
  mutate(label = factor(label, levels = region_order)) %>% 
ggplot(aes(x=label, y=est, ymin=CI_low, ymax=CI_up, colour=interaction(FDR,hemi))) +
        geom_pointrange(position = position_nudge(x = results$hemi_offset)) + 
        geom_hline(yintercept=0, lty=2) +  
        coord_flip() +
        xlab("") + ylab("Beta/95% CI") +
        theme_bw() + 
        facet_grid(~phenotype, scales='free') +
        scale_colour_manual(values=c("p >= 0.05.right" = "#F5C7B8FF", "p >= 0.05.left" = "#9FB1BCFF", 
                                     "p < 0.05.right" = "#e05022", "p < 0.05.left" = "#2E5266FF"),
                           labels = c("p>0.05 (Left)", 
                                  "p>0.05 (Right)",
                                  "p<0.05 (Left)",
                                  "p<0.05 (Right)")) +
        guides(colour = guide_legend(title = "")) +
        theme(legend.position="none", strip.background =element_rect(fill="white"),
              plot.margin = margin(t = 5, r = 20, b = 5, l = 5, unit = "pt")  ) + 
        scale_y_continuous(breaks = pretty(results$est, n = 1))
        #theme(legend.position="none",strip.background =element_rect(fill="white"), panel.spacing.x = unit(8, "mm"))

In [ ]:
results=results %>% mutate_if(is.numeric, round_and_format, digits = 4, threshold = 0.0001)
tab_noddi=flextable(results)
tab_noddi=autofit(tab_noddi, add_w = 0.1,add_h = 0.1,part = c("body", "header"),unit = "in")  %>% fit_to_width(max_width=8) %>% 
set_caption(caption = 'Table S6 Associations between subcortical NODDI measures and ultraprocessed food intake')

# Cognition

In [ ]:
data$RT=rowMeans(data[grep('snap.button', colnames(data))], na.rm=T) #reaction time
data$Pairs=rowMeans(data[grep('number_of_incorrect_matches_in_round', colnames(data))], na.rm=T) #visuospatial memory
data$Prospective_memory<-(as.numeric(factor(car::recode(data$prospective_memory_result_20018.2.0, 
                                         "'Correct recall on first attempt'='1'; 
                                        'Correct recall on second attempt'='2';
                                        'Instruction not recalled, either skipped or incorrect'='3'"), ordered=TRUE)))

cog=c('RT','Pairs','Prospective_memory','fluid_intelligence_score_20016.2.0',
      'maximum_digits_remembered_correctly_4282.2.0', 'number_of_puzzles_correct_21004.2.0') # 21004 - executive function

In [ ]:
results_cog=matrix(ncol=6,nrow=length(cog)) 
        
model=list()
model1=list()

        for (i in 1:length(cog)){ # cog is defined as list of cognitive variables at the beginning of the script

           model[[i]]=(lm(scale(data[[grep(cog[i], colnames(data))]]) ~ 
                      data$upf_perc +
                      data$saturated_fa +
                      data$total_sugar + 
                      data$sodium +
                      data$body_mass_index_bmi_21001.2.0 +
                      data$kJ_sum +
                      poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
                      data$sex_31.0.0 + 
                      poly(difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                               min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
                      data$uk_biobank_assessment_centre_54.2.0 + 
                      #data$qualifications_6138.2.0  +
                      data$smoking_status_20116.2.0 +
                      data$alcohol + 
                      #data$average_total_household_income_before_tax_738.2.0 +
                      data$number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0 +
                      data$number_of_daysweek_of_moderate_physical_activity_10._minutes_884.2.0
                                      ))

            model1[[i]]=summary(model[[i]])

            results_cog[i,1]=cog[i]
            results_cog[i,2]=model1[[i]]$coefficients[2,4] # p-value for UPF
            results_cog[i,3]=model1[[i]]$coefficients[2,3] # t-value for UPF
            results_cog[i,4]=model1[[i]]$coefficients[2,1] # estimate for 
            results_cog[i,5]=model1[[i]]$coefficients[2,2] # sd for 
            results_cog[i,6]=model1[[i]]$df[2]+model1[[i]]$df[1] # 
        }

        results_cog[,2]=p.adjust(results_cog[,2], method='BH')
        results_cog=as.data.frame(results_cog)
        colnames(results_cog)=c('Test','p','t', 'est','se', 'Sample size')
        results_cog$t=as.numeric(results_cog$t)
        results_cog$p=as.numeric(results_cog$p)
        results_cog$est=as.numeric(results_cog$est)
        results_cog$se=as.numeric(results_cog$se)

results_cog$label=c('Reaction time','Visuospatial memory','Prospective memory','Fluid intelligence',
      'Working memory', 'Executive function')

In [ ]:
results_cog=results_cog %>% mutate_if(is.numeric, round_and_format, digits = 4, threshold = 0.0001)
tab_cog=flextable(results_cog[,c(7,2:6)])
tab_cog=autofit(tab_cog, add_w = 0.1,add_h = 0.1,part = c("body", "header"),unit = "in")  %>% fit_to_width(max_width=8) %>% 
set_caption(caption = 'Table S7 Associations between cognition and ultraprocessed food intake')

In [ ]:
results_cog$FDR <- ifelse(results_cog$p < 0.05, "p < 0.05", "p >= 0.05")

png('/dagher/dagher11/filip/UPF/figures/Cognition.png', width=1500, height=1000, res=300)
results_cog %>%
  arrange(desc(est)) %>%    
  mutate(label=factor(label, levels=label)) %>%   
ggplot(aes(x=label, y=est, ymin=est-se, ymax=est+se, colour=FDR)) +
        geom_pointrange() + 
        geom_hline(yintercept=0, lty=2) +  # add a dotted line at x=0 after flip
        coord_flip() +  # flip coordinates (puts labels on y axis)
        xlab("Cognitive ability") + ylab("Beta/SE") +
        theme_bw() +# use a white background
        scale_colour_manual(values=c("p < 0.05" = "#ed563d", "p >= 0.05" = "#000000"))  # specify colours
dev.off()

# Follow-up analyses

## Mediation prerequisites

In [ ]:
corrs=c('c.reactive_protein_30710.0.0','triglycerides_30870.0.0', 
         'hdl_cholesterol_30760.0.0','glycated_haemoglobin_hba1c_30750.0.0')

brain=c('MNI_PD25_subcortical_LH_putamen.nii.gz.ICVF','MNI_PD25_subcortical_RH_putamen.nii.gz.ICVF',
        'MNI_PD25_subcortical_RH_nucleus_accumbens.nii.gz.ICVF','full_hypo_MD','globus_pallidus_LH.ICVF','globus_pallidus_RH.ICVF',
        'mean_fa_in_fornix_cres.stria_terminalis_on_fa_skeleton_left_25095.2.0',
       'MNI_PD25_subcortical_LH_thalamus.nii.gz.ISOVF','MNI_PD25_subcortical_LH_amygdala.nii.gz.ISOVF','right_nucleus_accumbens_MD',
       'mean_md_in_fornix_cres.stria_terminalis_on_fa_skeleton_right_25142.2.0','mean_fa_in_fornix_on_fa_skeleton_25061.2.0',
       'mean_fa_in_genu_of_corpus_callosum_on_fa_skeleton_25058.2.0', 'mean_thickness_of_lateraloccipital_left_hemisphere_27182.2.0',
       'mean_thickness_of_lateraloccipital_right_hemisphere_27275.2.0')

results=matrix(ncol=9,nrow=length(corrs)*length(brain))
index=0
for (i in 1:(length(brain))){
    for (j in 1:length(corrs)){
 
    
    m1=(lm(scale(data[[c(grep(corrs[j],colnames(data)))]]) ~ 
          scale(data[[c(grep(brain[i],colnames(data)))]]) +
          data$saturated_fa +
          data$total_sugar + 
          data$sodium +
          data$body_mass_index_bmi_21001.2.0 +
          data$Total_Energy_KJ +
          poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
          data$sex_31.0.0 + 
          poly(difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                   min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
          data$uk_biobank_assessment_centre_54.2.0 + 
          data$qualifications_6138.2.0  +
          data$smoking_status_20116.2.0 +
          data$alcohol + 
          data$average_total_household_income_before_tax_738.2.0 +
          data$number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0
                 ))

    sum_mod=summary(m1)
        
    results[j+(4*index),1]=brain[i]
    results[j+(4*index),2]=sum_mod$coefficients[2,4] # p-value 
    results[j+(4*index),3]=sum_mod$coefficients[2,3] # t-value 
    results[j+(4*index),4]=sum_mod$coefficients[2,1] # est for 
    results[j+(4*index),5]=sum_mod$coefficients[2,2] # se for 
    results[j+(4*index),6]=sum_mod$df[2]+sum_mod$df[1] #}  
    results[j+(4*index),7]=corrs[j]
    results[j+(4*index),8]=confint(m1)[2,1]
    results[j+(4*index),9]=confint(m1)[2,2]
  }
    
    index=index+1  
    
}


In [ ]:
# Assuming your dataframe is called 'results'
n_groups <- nrow(results) %/% 4  # Integer division to get number of complete groups

for(i in 1:n_groups) {
  start_idx <- (i-1)*4 + 1
  end_idx <- i*4
  results[start_idx:end_idx, 2] <- p.adjust(results[start_idx:end_idx, 2], method='BH')
}

results=as.data.frame(results)
colnames(results)=c('label','p','t','est','se','n','variable','CI_low','CI_up')
results$label=c(rep('Left Putamen ICVF',4),
                rep('Right Putamen ICFV',4),rep('Right Nucleus Accumbens ICVF',4),rep('Hypothalamus MD',4),
                rep('Left Globus Pallidus ICVF',4),rep('Right Globus Pallidus ICVF',4),rep('Left Fornix Cres Stria Terminalis FA',4),
                rep('Left Thalamus ISOVF',4),rep('Left Amygdala ISOVF',4),rep('Right Nucleus Accumbens MD',4),
               rep('Right Fornix Cres Stria Terminalis MD',4),rep('Fornix FA',4),rep('Genu of Corpus Callosum FA',4),
               rep('Left Lateral Occipital Cortex Thickness',4),rep('Right Lateral Occipital Cortex Thickness',4))
results$variable=rep(c('CRP','TG','HDL','HbA1c'),15)
results$t=as.numeric(results$t)
results$p=as.numeric(results$p)
results$se=as.numeric(results$se)
results$est=as.numeric(results$est)
results$CI_low=as.numeric(results$CI_low)
results$CI_up=as.numeric(results$CI_up)
colnames(results)=c('Parcel','p-value', 't-value', 'estimate', 'se', 'n', 'variable','CI_low','CI_up')
results_premed=results

In [ ]:
results_premed=results_premed %>% mutate_if(is.numeric, round_and_format, digits = 4, threshold = 0.0001)
tab_premed=flextable(results_premed[,c(1, 7, 2:6, 8:9)])
tab_premed=autofit(tab_premed, add_w = 0.1,add_h = 0.1,part = c("body", "header"),unit = "in")  %>% fit_to_width(max_width=8) %>% 
set_caption(caption = 'Table S12 Associations between brain phenotypes and chosen metabolic variables as a prerequisite to mediation analyses')

In [ ]:
png('/dagher/dagher11/filip/UPF/figures/Figure4.png', height=1000, width=1500, res=300)

plot_data <- results_premed %>%
  mutate(
    estimate = as.numeric(estimate),
    significance = ifelse(`p-value` < 0.05, "*", "")
  )

# Create the plot
ggplot(plot_data, aes(x = variable, y = Parcel)) +
  geom_tile(aes(fill = estimate), color = "gray", linewidth = 0.1) +
  geom_text(aes(label = significance), color = "black", size = 4) +
  scale_fill_gradient2(
    low = "#e05022",
    high = "#2E5266FF",
    mid = "white",
    midpoint = 0,
    name = "Beta"
  ) +
  theme_minimal() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1, size=9),
    axis.title = element_blank(),
    panel.grid = element_blank(),
    axis.text.y = element_text(size = 9),
  )

dev.off()

## Mediation

### Metabolic mediation


In [ ]:
extract_mediation_summary <- function (x) { 

  clp <- 100 * x$conf.level
  isLinear.y <- ((class(x$model.y)[1] %in% c("lm", "rq")) || 
                   (inherits(x$model.y, "glm") && x$model.y$family$family == 
                      "gaussian" && x$model.y$family$link == "identity") || 
                   (inherits(x$model.y, "survreg") && x$model.y$dist == 
                      "gaussian"))

  printone <- !x$INT && isLinear.y

  if (printone) {

    smat <- c(x$d1, x$d1.ci, x$d1.p)
    smat <- rbind(smat, c(x$z0, x$z0.ci, x$z0.p))
    smat <- rbind(smat, c(x$tau.coef, x$tau.ci, x$tau.p))
    smat <- rbind(smat, c(x$n0, x$n0.ci, x$n0.p))

    rownames(smat) <- c("ACME", "ADE", "Total Effect", "Prop. Mediated")

  } else {
    smat <- c(x$d0, x$d0.ci, x$d0.p)
    smat <- rbind(smat, c(x$d1, x$d1.ci, x$d1.p))
    smat <- rbind(smat, c(x$z0, x$z0.ci, x$z0.p))
    smat <- rbind(smat, c(x$z1, x$z1.ci, x$z1.p))
    smat <- rbind(smat, c(x$tau.coef, x$tau.ci, x$tau.p))
    smat <- rbind(smat, c(x$n0, x$n0.ci, x$n0.p))
    smat <- rbind(smat, c(x$n1, x$n1.ci, x$n1.p))
    smat <- rbind(smat, c(x$d.avg, x$d.avg.ci, x$d.avg.p))
    smat <- rbind(smat, c(x$z.avg, x$z.avg.ci, x$z.avg.p))
    smat <- rbind(smat, c(x$n.avg, x$n.avg.ci, x$n.avg.p))

    rownames(smat) <- c("ACME (control)", "ACME (treated)", 
                        "ADE (control)", "ADE (treated)", "Total Effect", 
                        "Prop. Mediated (control)", "Prop. Mediated (treated)", 
                        "ACME (average)", "ADE (average)", "Prop. Mediated (average)")

  }

  colnames(smat) <- c("Estimate", paste(clp, "% CI Lower", sep = ""), 
                      paste(clp, "% CI Upper", sep = ""), "p-value")
  smat

}

In [ ]:
mediators=c('c.reactive_protein_30710.0.0','triglycerides_30870.0.0', 
         'hdl_cholesterol_30760.0.0','glycated_haemoglobin_hba1c_30750.0.0')

VoI=c('MNI_PD25_subcortical_LH_putamen.nii.gz.ICVF','MNI_PD25_subcortical_RH_putamen.nii.gz.ICVF',
        'MNI_PD25_subcortical_RH_nucleus_accumbens.nii.gz.ICVF','full_hypo_MD','globus_pallidus_LH.ICVF','globus_pallidus_RH.ICVF',
        'mean_fa_in_fornix_cres.stria_terminalis_on_fa_skeleton_left_25095.2.0',
       'MNI_PD25_subcortical_LH_thalamus.nii.gz.ISOVF','MNI_PD25_subcortical_LH_amygdala.nii.gz.ISOVF','right_nucleus_accumbens_MD',
       'mean_md_in_fornix_cres.stria_terminalis_on_fa_skeleton_right_25142.2.0','mean_fa_in_fornix_on_fa_skeleton_25061.2.0',
       'mean_fa_in_genu_of_corpus_callosum_on_fa_skeleton_25058.2.0', 'mean_thickness_of_lateraloccipital_left_hemisphere_27182.2.0',
       'mean_thickness_of_lateraloccipital_right_hemisphere_27275.2.0')

# Create results matrix with enough rows for all combinations
results_med = matrix(nrow = length(mediators) * length(VoI), ncol = 19)

# Counter for row index
row_idx = 1

for (j in 1:length(VoI)) {
    for (i in 1:length(mediators)) {

    
    columns_to_check <- c("age_when_attended_assessment_centre_21003.2.0", 'body_mass_index_bmi_21001.2.0',
                         'sex_31.0.0', 'date_of_attending_assessment_centre_53.2.0','uk_biobank_assessment_centre_54.2.0',
                         'upf_perc','Total_Energy_KJ',                      
                         'qualifications_6138.2.0','alcohol','smoking_status_20116.2.0',
                         'number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0',
                         'average_total_household_income_before_tax_738.2.0', mediators[i], VoI[j])
    
    # Remove rows with NA values in the specified columns
    data_med <- data[complete.cases(data[, columns_to_check]), ]

    mediator_var = data_med[[mediators[i]]]
    outcome_var = data_med[[VoI[j]]]  # Get the current brain region variable
    
    med.fit <- lm(mediator_var ~ 
                  Percentage_NOVA_4 + 
                  saturated_fa +
                  total_sugar + 
                  sodium +
                  body_mass_index_bmi_21001.2.0 +
                  Total_Energy_KJ +
                  poly(age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
                  sex_31.0.0 + 
                  poly(difftime(as.Date(date_of_attending_assessment_centre_53.2.0), 
                           min(as.Date(date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
                  uk_biobank_assessment_centre_54.2.0 +
                  qualifications_6138.2.0  +
                  smoking_status_20116.2.0 +
                  alcohol + 
                  average_total_household_income_before_tax_738.2.0 +
                  number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0,
                  data = data_med)
    
    out.fit <- lm(outcome_var ~ 
                  mediator_var + 
                  Percentage_NOVA_4 +
                  saturated_fa +
                  total_sugar + 
                  sodium +
                  body_mass_index_bmi_21001.2.0 +
                  Total_Energy_KJ +
                  poly(age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
                  sex_31.0.0 + 
                  poly(difftime(as.Date(date_of_attending_assessment_centre_53.2.0), 
                           min(as.Date(date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
                  uk_biobank_assessment_centre_54.2.0 + 
                  qualifications_6138.2.0  +
                  smoking_status_20116.2.0 +
                  alcohol + 
                  average_total_household_income_before_tax_738.2.0 +
                  number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0, 
                  data = data_med)
    
    med.out <- mediate(med.fit, out.fit, treat = "Percentage_NOVA_4", mediator = 'mediator_var', robustSE = TRUE, sims = 100)

    med.sum = extract_mediation_summary(med.out)

    # Store results using the row counter
    results_med[row_idx, 1] = mediators[i]
    results_med[row_idx, 2] = VoI[j]  # Store the current brain region
    results_med[row_idx, 3] = med.sum[1,1]
    results_med[row_idx, 4] = med.sum[1,2]
    results_med[row_idx, 5] = med.sum[1,3]
    results_med[row_idx, 6] = med.sum[1,4]
    results_med[row_idx, 7] = med.sum[2,1]
    results_med[row_idx, 8] = med.sum[2,2]
    results_med[row_idx, 9] = med.sum[2,3]
    results_med[row_idx, 10] = med.sum[2,4]
    results_med[row_idx, 11] = med.sum[3,1]
    results_med[row_idx, 12] = med.sum[3,2]
    results_med[row_idx, 13] = med.sum[3,3]
    results_med[row_idx, 14] = med.sum[3,4]
    results_med[row_idx, 15] = med.sum[4,1]
    results_med[row_idx, 16] = med.sum[4,2]
    results_med[row_idx, 17] = med.sum[4,3]
    results_med[row_idx, 18] = med.sum[4,4]
    results_med[row_idx, 19] = nrow(data_med)


    # Increment row counter
    row_idx = row_idx + 1
    }
}

colnames(results_med) = c('Mediator','Region','ACME est', 'ACME CI low', 'ACME CI upp','ACME p-value',
                         'ADE est', 'ADE CI low', 'ADE CI upp','ADE p-value',
                         'Total est', 'Total CI low', 'Total CI upp','Total p-value',
                         'Proportion est', 'Proportion CI low', 'Proportion CI upp','Proportion p-value','Sample size')


results_med = as.data.frame(results_med)
results_med[,3:19] = sapply(results_med[,3:19], function(x) (as.numeric(x)))
results_med$Region=c(rep('Left Putamen ICVF',4),
                rep('Right Putamen ICFV',4),rep('Right Nucleus Accumbens ICVF',4),rep('Hypothalamus MD',4),
                rep('Left Globus Pallidus ICVF',4),rep('Right Globus Pallidus ICVF',4),rep('Left Fornix Cres Stria Terminalis FA',4),
                rep('Left Thalamus ISOVF',4),rep('Left Amygdala ISOVF',4),rep('Right Nucleus Accumbens MD',4),
               rep('Right Fornix Cres Stria Terminalis MD',4),rep('Fornix FA',4),rep('Genu of Corpus Callosum FA',4),
               rep('Left Lateral Occipital Cortex Thickness',4),rep('Right Lateral Occipital Cortex Thickness',4))
results_med$Mediator=rep(c('CRP','TG','HDL','HbA1c'),15)
results_med1 = results_med

In [ ]:
results_med1 <- results_med1[results_premed$`p-value` < 0.05, ]

In [ ]:
a=plot_mediation_results(results_med1, 1)

In [ ]:
mediators=c('body_mass_index_bmi_21001.2.0')

VoI=c('MNI_PD25_subcortical_LH_putamen.nii.gz.ICVF','MNI_PD25_subcortical_RH_putamen.nii.gz.ICVF',
        'MNI_PD25_subcortical_RH_nucleus_accumbens.nii.gz.ICVF','full_hypo_MD','globus_pallidus_LH.ICVF','globus_pallidus_RH.ICVF',
        'mean_fa_in_fornix_cres.stria_terminalis_on_fa_skeleton_left_25095.2.0',
       'MNI_PD25_subcortical_LH_thalamus.nii.gz.ISOVF','MNI_PD25_subcortical_LH_amygdala.nii.gz.ISOVF','right_nucleus_accumbens_MD',
       'mean_md_in_fornix_cres.stria_terminalis_on_fa_skeleton_right_25142.2.0','mean_fa_in_fornix_on_fa_skeleton_25061.2.0',
       'mean_fa_in_genu_of_corpus_callosum_on_fa_skeleton_25058.2.0', 'mean_thickness_of_lateraloccipital_left_hemisphere_27182.2.0',
       'mean_thickness_of_lateraloccipital_right_hemisphere_27275.2.0')

# Create results matrix with enough rows for all combinations
results_med = matrix(nrow = length(mediators) * length(VoI), ncol = 19)

# Counter for row index
row_idx = 1

for (j in 1:length(VoI)) {
    for (i in 1:length(mediators)) {

    
    columns_to_check <- c("age_when_attended_assessment_centre_21003.2.0", 'body_mass_index_bmi_21001.2.0',
                         'sex_31.0.0', 'date_of_attending_assessment_centre_53.2.0','uk_biobank_assessment_centre_54.2.0',
                         'upf_perc','Total_Energy_KJ',                      
                         'qualifications_6138.2.0','alcohol','smoking_status_20116.2.0',
                         'number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0',
                         'average_total_household_income_before_tax_738.2.0', mediators[i], VoI[j])
    
    # Remove rows with NA values in the specified columns
    data_med <- data[complete.cases(data[, columns_to_check]), ]

    mediator_var = data_med[[mediators[i]]]
    outcome_var = data_med[[VoI[j]]]  # Get the current brain region variable
    
    med.fit <- lm(mediator_var ~ 
                  Percentage_NOVA_4 + 
                  saturated_fa +
                  total_sugar + 
                  sodium +
                  Total_Energy_KJ +
                  poly(age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
                  sex_31.0.0 + 
                  poly(difftime(as.Date(date_of_attending_assessment_centre_53.2.0), 
                           min(as.Date(date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
                  uk_biobank_assessment_centre_54.2.0 +
                  qualifications_6138.2.0  +
                  smoking_status_20116.2.0 +
                  alcohol + 
                  average_total_household_income_before_tax_738.2.0 +
                  number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0,
                  data = data_med)
    
    out.fit <- lm(outcome_var ~ 
                  mediator_var + 
                  Percentage_NOVA_4 +
                  saturated_fa +
                  total_sugar + 
                  sodium +
                  Total_Energy_KJ +
                  poly(age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
                  sex_31.0.0 + 
                  poly(difftime(as.Date(date_of_attending_assessment_centre_53.2.0), 
                           min(as.Date(date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
                  uk_biobank_assessment_centre_54.2.0 + 
                  qualifications_6138.2.0  +
                  smoking_status_20116.2.0 +
                  alcohol + 
                  average_total_household_income_before_tax_738.2.0 +
                  number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0, 
                  data = data_med)
    
    med.out <- mediate(med.fit, out.fit, treat = "Percentage_NOVA_4", mediator = 'mediator_var', robustSE = TRUE, sims = 100)

    med.sum = extract_mediation_summary(med.out)

    # Store results using the row counter
    results_med[row_idx, 1] = mediators[i]
    results_med[row_idx, 2] = VoI[j]  # Store the current brain region
    results_med[row_idx, 3] = med.sum[1,1]
    results_med[row_idx, 4] = med.sum[1,2]
    results_med[row_idx, 5] = med.sum[1,3]
    results_med[row_idx, 6] = med.sum[1,4]
    results_med[row_idx, 7] = med.sum[2,1]
    results_med[row_idx, 8] = med.sum[2,2]
    results_med[row_idx, 9] = med.sum[2,3]
    results_med[row_idx, 10] = med.sum[2,4]
    results_med[row_idx, 11] = med.sum[3,1]
    results_med[row_idx, 12] = med.sum[3,2]
    results_med[row_idx, 13] = med.sum[3,3]
    results_med[row_idx, 14] = med.sum[3,4]
    results_med[row_idx, 15] = med.sum[4,1]
    results_med[row_idx, 16] = med.sum[4,2]
    results_med[row_idx, 17] = med.sum[4,3]
    results_med[row_idx, 18] = med.sum[4,4]
    results_med[row_idx, 19] = nrow(data_med)


    # Increment row counter
    row_idx = row_idx + 1
    }
}

colnames(results_med) = c('Mediator','Region','ACME est', 'ACME CI low', 'ACME CI upp','ACME p-value',
                         'ADE est', 'ADE CI low', 'ADE CI upp','ADE p-value',
                         'Total est', 'Total CI low', 'Total CI upp','Total p-value',
                         'Proportion est', 'Proportion CI low', 'Proportion CI upp','Proportion p-value','Sample size')


results_med = as.data.frame(results_med)
results_med[,3:19] = sapply(results_med[,3:19], function(x) (as.numeric(x)))
results_med$Region=c(rep('Left Putamen ICVF',1),
                rep('Right Putamen ICFV',1),rep('Right Nucleus Accumbens ICVF',1),rep('Hypothalamus MD',1),
                rep('Left Globus Pallidus ICVF',1),rep('Right Globus Pallidus ICVF',1),rep('Left Fornix Cres Stria Terminalis FA',1),
                rep('Left Thalamus ISOVF',1),rep('Left Amygdala ISOVF',1),rep('Right Nucleus Accumbens MD',1),
               rep('Right Fornix Cres Stria Terminalis MD',1),rep('Fornix FA',1),rep('Genu of Corpus Callosum FA',1),
               rep('Left Lateral Occipital Cortex Thickness',1),rep('Right Lateral Occipital Cortex Thickness',1))
results_med$Mediator=rep(c('BMI'),15)
results_med2 = results_med

In [ ]:
results_med=rbind(results_med1, results_med2)

In [ ]:
results_med[,6]=p.adjust(results_med[,6],method='BH')
results_med[,10]=p.adjust(results_med[,10],method='BH')
results_med[,14]=p.adjust(results_med[,14],method='BH')
results_med[,18]=p.adjust(results_med[,18],method='BH')

In [ ]:
results_med=results_med %>% mutate_if(is.numeric, round_and_format, digits = 4, threshold = 0.0001)
tab_med=flextable(results_med)
tab_med=autofit(tab_med, add_w = 0.1,add_h = 0.1,part = c("body", "header"),unit = "in")  %>% fit_to_width(max_width=9) %>% 
set_caption(caption = 'Table S13 Mediation analyses results')

## Old stuff

In [ ]:
taq=read.table('~/Downloads/UKBB_Taq_coded.csv', header=T, sep=',')

In [ ]:
data=merge(data, taq, by.x='eid', by.y='FID', all.x=T)

In [ ]:
unique(data$Taq1A)

In [ ]:
data$Taq1A_rec=dplyr::recode(data$Taq1A, 'A1A1'=2, 'A1A2'=1,'A2A2'=0)

In [ ]:
(summary(lm(data$full_hypo_MD~ 
          data$Taq1A_rec *
          data$upf_perc +
          data$saturated_fa +
          data$total_sugar + 
          data$sodium +
          data$body_mass_index_bmi_21001.2.0 +
          poly(data$age_when_attended_assessment_centre_21003.2.0, 2, raw=TRUE) *
          data$sex_31.0.0 + 
          poly(difftime(as.Date(data$date_of_attending_assessment_centre_53.2.0), 
                   min(as.Date(data$date_of_attending_assessment_centre_53.2.0)), units='days'), 2, raw=TRUE) +
          data$uk_biobank_assessment_centre_54.2.0 + 
          data$qualifications_6138.2.0  +
          data$smoking_status_20116.2.0 +
          data$alcohol + 
          data$average_total_household_income_before_tax_738.2.0 +
          data$number_of_daysweek_of_vigorous_physical_activity_10._minutes_904.2.0)))$coefficients

In [ ]:
ggplot(data=data, aes(x=as.factor(Taq1A_rec), y=data$`body_mass_index_bmi_21001.2.0`)) + geom_boxplot()

# Save tables

In [ ]:
save_as_docx(tab_corr, tab_cortex, tab_subcortex, tab_wm, tab_dti, tab_noddi, tab_premed, tab_med,
             path='/dagher/dagher11/filip/UPF/tables/All_tables_November.docx')

In [ ]:
save_flextables_to_xlsx(
  flextable_list = list(tab_corr, tab_cortex, tab_subcortex, tab_wm, tab_dti, tab_noddi),#, tab_premed, tab_med),
  output_path = '/dagher/dagher11/filip/UPF/tables/All_tables_November_AIC.xlsx'
)

# Plotting

In [ ]:
common_theme <- theme(
  axis.text.x = element_text(size = 12),
  axis.text.y = element_text(size = 15),
  legend.text = element_text(size = 12)
)

In [ ]:
png('/dagher/dagher11/filip/UPF/figures/Figure2_revision.png', height=3000, width=5000, res=300)
ggarrange(forest_CTSA + common_theme,
          ggarrange(forest_subvol + common_theme, forest_WM + common_theme, 
                    labels=c('b','c'),  font.label = list(size = 15, color = "black"), ncol=1, nrow=2, heights=c(1,1.5)),
          labels = c("a", ""), font.label = list(size = 15, color = "black"),
          ncol = 2, nrow = 1, widths=c(1.1,1)) +
          theme(plot.margin = margin(0,0,0,0, "cm"))
dev.off()

In [ ]:
png('/dagher/dagher11/filip/UPF/figures/Figure3_revision.png', height=1700, width=5000, res=300)
ggarrange(forest_subDTI + common_theme, forest_subNODDI + common_theme,
          labels = c("a", "b"), font.label = list(size = 15, color = "black"),
          ncol = 2, nrow = 1, widths=c(1.2,1)) +
          theme(plot.margin = margin(0,0,0,0, "cm"))
dev.off()